<a href="https://colab.research.google.com/github/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/blob/main/Sentiment_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd

In [11]:
fname="https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5f1b18584a67905e021e1f495619a124247a5330/Train_all_sentence.fth"


In [12]:
!wget $fname

--2021-10-05 20:57:01--  https://github.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/raw/5f1b18584a67905e021e1f495619a124247a5330/Train_all_sentence.fth
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5f1b18584a67905e021e1f495619a124247a5330/Train_all_sentence.fth [following]
--2021-10-05 20:57:02--  https://raw.githubusercontent.com/sipocz/Twitter-Sentiment-Analysis-AI-Challenge/5f1b18584a67905e021e1f495619a124247a5330/Train_all_sentence.fth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2951338 (2.8M) [application/octet-stream]
Saving to: ‘

In [13]:
df=pd.read_feather("Train_all_sentence.fth")

In [14]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text,sentece_code
0,570018445695148033,negative,1.0,Customer Service Issue,0.6560,United,None,tpensari,None,0,@united your customer service is terrible! Sto...,None,2015-02-23 16:32:42 -0800,None,None,your customer service is terrible stood inline...,"[8325, 1798, 6541, 3856, 7342, 7023, 3734, 350..."
1,568830255282040832,neutral,1.0,None,NaN,Southwest,None,leavenodoubt131,None,0,@SouthwestAir I will dm you now,None,2015-02-20 09:51:15 -0800,None,Tijuana,i will do you now,"[3551, 8173, 2175, 8319, 4965]"
2,569296356143050754,negative,1.0,longlines,0.3462,United,None,ScottyNeuman,None,1,@united congratulations united. Fail again. Sk...,"[39.85760174, -104.66669625]",2015-02-21 16:43:22 -0800,"Montana, God's country\n",Pacific Time (US & Canada),congratulations united fail again sky at a gat...,"[1562, 7772, 2657, 153, 6712, 502, 1, 3029, 82..."
3,569936640669581313,positive,1.0,None,NaN,United,None,koploperfan1992,None,0,@united look at this beauty 😉 dc 10 united air...,"[51.322819, 5.3576218]",2015-02-23 11:07:38 -0800,Winterfell,None,look at this beauty do united airlines,"[4323, 502, 7402, 694, 2175, 7772, 197]"
4,569542242983124992,positive,1.0,None,NaN,Delta,None,sandyy889,None,0,@JetBlue Landing! As usual great flight wiyh a...,None,2015-02-22 09:00:26 -0800,None,America/Detroit,landing as usual great flight wiyh a great cre...,"[4092, 457, 7871, 3170, 2829, 8213, 1, 3170, 1..."


In [15]:
table=list(df["sentece_code"]) # typo
lens=max([len(x) for x in table])
sentiment=list(df["airline_sentiment"])


In [16]:
ytrain=[]
for sent in sentiment:
    if sent=="negative":
        o=[1,0,0]
    if sent=="neutral":
        o=[0,1,0]
    if sent=="positive":
        o=[0,0,1]
    ytrain.append(o)
    

In [17]:
x0=[0 for _ in range(lens) ]
xtrain=[]
for sent in table:
    o1=list(x0[0:lens-len(sent)]+list(sent))
    xtrain.append(o1)

In [18]:
print(lens)

33


In [19]:


lstm_size=lens
max_input_length=lens
embedding_size=100
n_words=len(table)

In [20]:

# Importáld a megfelelő rétegeket
from tensorflow.keras.layers import Input,Dense,Embedding,LSTM,TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adadelta,Adam,SGD
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy
from tensorflow.keras import backend as K
# Gondolj bele, hogy a hálóban az első réteg egy "beágazás" kell majd legyen
# Ne feledd behozni a funkcionális vagy szekvenciális API-nak megfelelő "fő" osztályt
# Adott esetben az optimalizálót
# Nem külömben a "bakcendet", hogy jó gyakorlat szerint reseteld a gráfot
#tf.reset_default_graph()
K.clear_session
x= Input(shape=(max_input_length,))
embedded_x=Embedding(n_words+1,embedding_size, input_length=max_input_length-1, mask_zero=True)(x)
lstm_output=LSTM(lstm_size,return_sequences=True)(embedded_x)
lstm_output=LSTM(lstm_size,return_sequences=False)(lstm_output)
predictions= Dense(3, activation="softmax")(lstm_output)
model=Model(inputs=x, outputs=predictions)



In [21]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 33)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 33, 100)           1200100   
_________________________________________________________________
lstm (LSTM)                  (None, 33, 33)            17688     
_________________________________________________________________
lstm_1 (LSTM)                (None, 33)                8844      
_________________________________________________________________
dense (Dense)                (None, 3)                 102       
Total params: 1,226,734
Trainable params: 1,226,734
Non-trainable params: 0
_________________________________________________________________


In [22]:
x

<KerasTensor: shape=(None, 33) dtype=float32 (created by layer 'input_1')>

In [23]:
import numpy as np

In [24]:
x_train=xtrain[0:-2000]
y_train=ytrain[0:-2000]
x_test=xtrain[-2000:]
y_test=ytrain[-2000:]

In [25]:
train_x = np.asarray(xtrain)
train_y = np.asarray(ytrain)

In [26]:
# Loss 

loss = categorical_crossentropy # One-hot enkódolt kimenetünk van. Mit is használunk?

# Optimizer
optimizer = Adam() #Ízlés szerint...
 
# Compilation
#############

model.compile(optimizer=optimizer,loss=loss)

In [ ]:
# Illesszük az adatra a modellt. Használjunk 10% validációt
history=model.fit(x=xtrain,y=ytrain,validation_data=( x_test,y_test), epochs=25, batch_size=100)
# - nyelvmodellnél ez nem olyan lényeges
# Használhatjuk a Keras beépített validációs splitjét.
# Adjunk meg reális batch méretet!

Epoch 1/25
120/120 [==============================] - 9s 72ms/step - loss: 0.0444 - val_loss: 0.0397
Epoch 2/25
120/120 [==============================] - 8s 70ms/step - loss: 0.0414 - val_loss: 0.0353
Epoch 3/25
120/120 [==============================] - 8s 70ms/step - loss: 0.0414 - val_loss: 0.0332
Epoch 4/25
120/120 [==============================] - 8s 70ms/step - loss: 0.0392 - val_loss: 0.0324
Epoch 5/25
120/120 [==============================] - 8s 70ms/step - loss: 0.0318 - val_loss: 0.0305
Epoch 6/25
120/120 [==============================] - 8s 70ms/step - loss: 0.0353 - val_loss: 0.0262
Epoch 7/25
120/120 [==============================] - 8s 69ms/step - loss: 0.0287 - val_loss: 0.0241
Epoch 8/25
120/120 [==============================] - 8s 70ms/step - loss: 0.0267 - val_loss: 0.0244
Epoch 9/25
120/120 [==============================] - 8s 69ms/step - loss: 0.0292 - val_loss: 0.0264
Epoch 10/25
120/120 [==============================] - 8s 70ms/step - loss: 0.0294 - val_lo